# This notebook perform fault slip analysis (FSA) based on principal stresses from CMG geomechanical simulations.

# 1. Extract grid coordinates and fault id from Petrel exported files. Only need to do once for a 3D grid in Petrel. Save it as numpy array for later use.

# 2. Extract principla stresses (STRESMXP, STRESMNP, STRESINT) from CMG simulation results (gmch.sr3 -> rwo -> numpy).

# 3. Peform fault slip analysis

## perform fault slip analysis on each case

In [2]:
from fault_slip_analysis import FSA_stress_based

for case_num in range(1, 3):
    FSA_stress_based(
        stress_folder_path = "data/results_trimmed_gmc",
        parameter_file_path = "data/250915_CMG_parameters.csv",
        save_folder_path = "results/fault_slip_gmc",
        case_name = f"case{case_num}", 
        )

Processing case1...
Processing case2...


/Users/jijuid/Library/CloudStorage/GoogleDrive-jihuid@stanford.edu/My Drive/Github/postCMGsim/fault_slip_analysis.py:124: RuntimeWarning: divide by zero encountered in divide
  fault_slip = ((tau - cohesion) / sigma >= mu).astype(np.int8)
/Users/jijuid/Library/CloudStorage/GoogleDrive-jihuid@stanford.edu/My Drive/Github/postCMGsim/fault_slip_analysis.py:124: RuntimeWarning: divide by zero encountered in divide
  fault_slip = ((tau - cohesion) / sigma >= mu).astype(np.int8)


In [ ]:
import numpy as np

stress_folder_path = "data/results_trimmed_gmc"
case_name = "case1"
SH_stress = np.load(f'{stress_folder_path}/{case_name}_STRESMXP.npy')
Sh_stress = np.load(f'{stress_folder_path}/{case_name}_STRESMNP.npy')
Sv_stress = np.load(f'{stress_folder_path}/{case_name}_STRESINT.npy')

print(np.mean(SH_stress))
print(Sh_stress.shape)
print(Sv_stress.shape)

my_array = Sv_stress
contains_zero = np.any(my_array == 0)
print(f"Does the array contain zero? {contains_zero}")
num_zeros = np.count_nonzero(my_array == 0)
print(f"Number of zeros: {num_zeros}")
print(num_zeros/(107*117*5*6))
# Calculate basic statistics for all elements
mean_val = np.mean(my_array)
median_val = np.median(my_array)
std_dev = np.std(my_array)
min_val = np.min(my_array)
max_val = np.max(my_array)
sum_val = np.sum(my_array)
print(f"Mean: {mean_val}")
print(f"Median: {median_val}")
print(f"Standard Deviation: {std_dev:.2f}")
print(f"Min: {min_val}")
print(f"Max: {max_val}")
print(f"Sum: {sum_val}")

11645.463173576165
(107, 117, 5, 6)
(107, 117, 5, 6)
Does the array contain zero? True
Number of zeros: 174600
0.4648933620896238
Mean: 8602.07771991906
Median: 5434.495
Standard Deviation: 9473.23
Min: 0.0
Max: 33950.8
Sum: 3230682329.270001


## calculate fault slip percentage on each case

In [12]:
import numpy as np

n_cases = 90
n_time = 6
fault_slip_mean = np.zeros((n_cases,n_time))
for case_num in range(1, n_cases+1):
    fault_slip = np.load(f"results/fault_slip3/case{case_num}_fault_slip.npy")
    fault_slip_mean[case_num-1,:] = np.mean(fault_slip,axis=(0,1,2))

# save as csv
np.savetxt("results/fault_slip_mean3.csv",fault_slip_mean,delimiter=",",fmt="%.4f")
# save for DGSA responses
# year = 2050; year_list = [2030, 2040, 2050, 2060, 2550, 3050]
# np.savetxt("results/250819_CMG_responses2.csv",fault_slip_mean[:,year_list.index(year)],delimiter=",",fmt="%.4f")

## calculate fault slip percentage on a specific fault at a specific time

In [3]:
import numpy as np
# parameters
fault_id = 0; year = 2050; year_list = [2030, 2040, 2050, 2060, 2550, 3050]
# load data
coor_fault = np.load('results/JD_Sula_2025_flow_coor&fault_reservoir.npy')
fault_slip = np.load('results/fault_slip/case1_fault_slip.npy')

# Filter data for the specific fault_id
mask = (coor_fault[:,:,:,3] == fault_id)
fault_slip_val = fault_slip[:,:,:,year_list.index(year)][mask]
print(np.mean(fault_slip_val))

0.44986449864498645
